# Notebook to store neighborhoods records (JSON) into bronze.neighborhoods table

Define constant variables (location, target entity)

In [0]:
%run ../../config/utils

In [0]:
TARGET_TABLE="brz_neighborhoods"

### Getting the neighborhoods (parquet)

In [0]:
df_neighborhoods=spark.read.parquet(neighborhoods_volume_path)
df_neighborhoods.createOrReplaceTempView("neighborhoods")

### Write into bronze.neighborhoods

In [0]:
spark.sql(f"""
  MERGE INTO {catalog_name}.{bronze_schema_name}.{TARGET_TABLE} AS target
  USING neighborhoods AS source
  ON target.OBJECTID = source.OBJECTID
  WHEN MATCHED AND (
      target.CODIGO <> source.CODIGO OR
      target.NOMBRE <> source.NOMBRE OR
      target.IDENTIFICACION <> source.IDENTIFICACION OR
      target.LIMITEMUNICIPIOID <> source.LIMITEMUNICIPIOID OR
      target.SUBTIPO_COMUNACORREGIMIENTO <> source.SUBTIPO_COMUNACORREGIMIENTO OR
      target.LINK_DOCUMENTO <> source.LINK_DOCUMENTO OR
      target.SHAPEAREA <> source.SHAPEAREA OR
      target.SHAPELEN <> source.SHAPELEN OR
      target.geometry <> source.geometry
  ) THEN
    UPDATE SET
      target.CODIGO = source.CODIGO,
      target.NOMBRE = source.NOMBRE,
      target.IDENTIFICACION = source.IDENTIFICACION,
      target.LIMITEMUNICIPIOID = source.LIMITEMUNICIPIOID,
      target.SUBTIPO_COMUNACORREGIMIENTO = source.SUBTIPO_COMUNACORREGIMIENTO,
      target.LINK_DOCUMENTO = source.LINK_DOCUMENTO,
      target.SHAPEAREA = source.SHAPEAREA,
      target.SHAPELEN = source.SHAPELEN,
      target.geometry = source.geometry
  WHEN NOT MATCHED THEN
    INSERT (
      OBJECTID,
      CODIGO,
      NOMBRE,
      IDENTIFICACION,
      LIMITEMUNICIPIOID,
      SUBTIPO_COMUNACORREGIMIENTO,
      LINK_DOCUMENTO,
      SHAPEAREA,
      SHAPELEN,
      geometry
    )
    VALUES (
      source.OBJECTID,
      source.CODIGO,
      source.NOMBRE,
      source.IDENTIFICACION,
      source.LIMITEMUNICIPIOID,
      source.SUBTIPO_COMUNACORREGIMIENTO,
      source.LINK_DOCUMENTO,
      source.SHAPEAREA,
      source.SHAPELEN,
      source.geometry
    )
""")